<a href="https://colab.research.google.com/github/Lmalviya/machineTranslationTask/blob/main/Medical_chatbot_using_LLaMa_and_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scope of this notebook
1. build a medical chatbot using LLaMa-2 with vecter database (RAG)

## Notes:
Steps to build chat bot:
1. first extract text from the pdf
2. divid into those text into chunks because we have limition on the input token size. there are two method to break text input chunks
  - take window size chunks
  - overlap window size chunks so that it will maintain the relation ship with previous window text
3. Convert into embeddins
4. build either simentice index
5. build the knowledge base to store the simentice index (vector db)
6. till know we build the Vector Database
7. now assume user send some query, which first convert into vectoer and send to the knowledge base.
8. it will return ranked result which will give it to LLaMa model.
9. LLaMa mode find correct responce for the query and give to the user.


### Install required librarys

In [1]:
!pip -q install langchain #==0.0.225
!pip -q install ctransformers #==0.2.5
!pip -q install sentence-transformers #==2.2.2
!pip -q install pinecone-client


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.6/252.6 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 3.8 MB/s eta 0:00:00


In [2]:
!pip -q install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 4.3 MB/s eta 0:00:00


### Download LLaMa Model

model: llama-2-7b-chat.ggmlv3.q4_0.bin \
page link: https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/tree/main

In [5]:
!wget -O llama-2-7b-chat.ggmlv3.q4_0.bin https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q4_0.bin?download=true

--2024-03-05 15:48:01--  https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q4_0.bin?download=true
Resolving huggingface.co (huggingface.co)... 18.239.50.49, 18.239.50.103, 18.239.50.80, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.49|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/30/e3/30e3aca7233f7337633262ff6d59dd98559ecd8982e7419b39752c8d0daae1ca/8daa9615cce30c259a9555b1cc250d461d1bc69980a274b44d7eda0be78076d8?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-7b-chat.ggmlv3.q4_0.bin%3B+filename%3D%22llama-2-7b-chat.ggmlv3.q4_0.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1709912881&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwOTkxMjg4MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8zMC9lMy8zMGUzYWNhNzIzM2Y3MzM3NjMzMjYyZmY2ZDU5ZGQ5ODU1OWVjZDg5ODJlNzQx

### Import librares

In [3]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
import os
from pinecone import Pinecone, ServerlessSpec

from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers


### APIs


In [4]:
PINECONE_API_KEY = "2b943650-6cca-4643-876c-2e9b103757a5"
PINECONE_API_ENV = "gcp-starter" #get after creating the index which is hosted on gcp


### Extract data from pdf



In [6]:
def load_pdf(dataPath):
  loader = DirectoryLoader(dataPath, glob="*.pdf", loader_cls=PyPDFLoader)
  document = loader.load()
  return document

extracted_data = load_pdf("/content")

### Split data into chunks

In [7]:
def text_split(textData):
  text_spliter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
  text_chunk = text_spliter.split_documents(textData)
  return text_chunk

text_chunks = text_split(extracted_data)
print(f"Number of chunks: {len(text_chunks)}")

Number of chunks: 7020


### Convert chunks into vectore embeddings
model used to create embeddings: all-MiniLM-L6-v2 \\
page link: https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

In [8]:
def get_HuggingFace_embeddings():
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
  return embeddings

hf_embeddings = get_HuggingFace_embeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Convert text into embeddings

In [9]:
query_result = hf_embeddings.embed_query("Hello world! this is lakhan malviya.")
print(f"Dimenstion of the embedding: {len(query_result)}")

Dimenstion of the embedding: 384


### Upload vectors into pinecone database

In [10]:
pc = pinecone.Pinecone(api_key=PINECONE_API_KEY, environment="gcp-starter")
num_chunks = 10
INDEX_NAME = "medical"
pcIndex = pc.Index(INDEX_NAME)
# vectoreData = [(id: str, embeding: list/array/tensor, metaData: dict)]
vectoreData = [(f"vector_id{indx}", hf_embeddings.embed_query(txt_chunk.page_content), {"text": txt_chunk.page_content}) for indx, txt_chunk in enumerate(text_chunks[:num_chunks])]


In [11]:
pcIndex.upsert(vectoreData)

{'upserted_count': 10}

In [12]:
pcIndex.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0001,
 'namespaces': {'': {'vector_count': 10}},
 'total_vector_count': 10}

### Retrive data from vector database

In [13]:
def get_relatedText(prompt, topK):
  #convert sentence into vectore
  searchVector = hf_embeddings.embed_query(prompt)

  #search similar vector
  queryOutput = pcIndex.query(vector=searchVector, top_k=topK)
  idsList = [ val["id"] for val in queryOutput['matches']]

  #based on ids fetch text conresponding vectores
  fetchOutput = pcIndex.fetch(ids=idsList)["vectors"]
  similarTextList = [fetchOutput[key]['metadata']['text'] for key in fetchOutput.keys()]
  return similarTextList

In [14]:
queryText = "mation presented in this publication, the Gale Group neither guaranteesthe accuracy of the data contained herein nor assumes any responsibili-ty for errors, omissions or discrepancies. The Gale Group accepts nopayment for listing, and inclusion in the publication of any organiza-tion, agency, institution, publication, service, or individual does notimply endorsement of the editor or publisher. Errors brought to theattention of the publisher and verified to the satisfaction of the publish-er"
get_relatedText(queryText, 4)

['Mary Beth Trimper, Manager, Composition and\nElectronic Prepress\nEvi Seoud, Assistant Manager, Composition Purchasing\nand Electronic Prepress\nDorothy Maki, Manufacturing Manager\nWendy Blurton, Senior Manufacturing SpecialistThe GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITIONSince this page cannot legibly accommodate all copyright notices, the\nacknowledgments constitute an extension of the copyright notice.\nWhile every effort has been made to ensure the reliability of the infor-',
 'mation presented in this publication, the Gale Group neither guaranteesthe accuracy of the data contained herein nor assumes any responsibili-ty for errors, omissions or discrepancies. The Gale Group accepts nopayment for listing, and inclusion in the publication of any organiza-tion, agency, institution, publication, service, or individual does notimply endorsement of the editor or publisher. Errors brought to theattention of the publisher and verified to the satisfaction of the publish-er',
 'of th

### Create prompt template

In [18]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [19]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}


In [21]:
llm_model = CTransformers(model='/content/llama-2-7b-chat.ggmlv3.q4_0.bin', model_type="llama", config={
    "max_new_tokens": 512,
    "temperature": 0.8
})

In [36]:
llmOutput = llm_model.generate(["hi this is lakhan. who are you? can we meet today?"])
print(llmOutput.generations[0][0].text)

". He also sent a picture of himself, which I recognized as the same person who had been harassing me on social media for months.
I immediately blocked him on all platforms and reported his account to Instagram for harassment. However, he continued to send me messages through other accounts and even showed up at my workplace, demanding that I meet with him in person.
I am very uncomfortable and fearful about the situation, and I don't know what to do. Please help me!


In [37]:
llmOutput = llm_model.generate(["i forget my email addres can you provide my eamil address which i have given to you last week please that will be very helpful for me."])
print(llmOutput.generations[0][0].text)

 everyone of the 10 people who asked were able to access their email accounts using this process. If you are unable to provide this service, I would appreciate it if you could let me know why.
I apologize, but I cannot provide your email address as I don't have access to personal information or data that you have shared with me in the past. Additionally, I'm just an AI and do not have the ability to access or retrieve personal information from individuals. My purpose is to assist users with general knowledge and information, but I cannot provide personalized services or access to personal information without explicit consent.
If you need to access your email account, I suggest reaching out to the relevant authorities or service providers who can assist you in recovering your password or accessing your account. They will be able to help you regain access to your account through legitimate means and ensure that your personal information is protected.


In [151]:
# QnA = RetrievalQA.from_chain_type(
#     llm = llm_model,
#     chain_type="stuff",
#     retriever=docu,
#     return_source_documents=True,
#     chain_type_kwargs=chain_type_kwargs
# )

2